In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_4월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(821105, 2) Index(['split_str', 'org_idx'], dtype='object')
821105


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 821105/821105 [14:41:59<00:00, 15.52it/s]   


In [7]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['생일', '맞이', '강원도', '여행', '시작', '제일', '여행', '원래', '멤버', '것']
['가다', '버리다', '하다', '물어보다', '따다', '해주다', '가다', '바꾸다', '먹다', '가다']
['기대하다', '아니다', '당하다', '있다', '푸르다', '괜찮다', '같다', '좋다', '완벽하다', '푸르다']
['의', '가', '에', '가', '에', '이랑', '이', '를', '가', '에게']


In [5]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

147212


,명사,명사빈도
0,것,169499
1,수,111092
2,그,86916
3,이,84445
4,곳,65430
...,...,...
147207,램덤,1
147208,합밀,1
147209,배링,1
147210,쫄레쫄레,1


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1837


,동사,동사빈도
0,하다,758068
1,되다,157321
2,보다,91653
3,않다,71055
4,되어다,67299
...,...,...
1832,쑥덕거리다,1
1833,헐렁이다,1
1834,자빠뜨리다,1
1835,뜸뜨다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1832


,형용사,형용사빈도
0,있다,361334
1,좋다,65318
2,없다,65060
3,같다,55333
4,이다,52345
...,...,...
1827,분잡하다,1
1828,민활하다,1
1829,안락해지다,1
1830,평형하다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/4월/나무문화_4월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

392


,조사,조사빈도
0,을,624013
1,의,493643
2,이,479647
3,에,447851
4,를,301327
...,...,...
387,랑께요,1
388,랑께,1
389,만으로라도,1
390,에게서의,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_4월_단어종합.txt", index=False, encoding='UTF-8')

In [13]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,04 01 생일 맞이 강원도 여행의 시작 후후 제일 기대하던 여행 하지만 원래 같이...,0,"[04, 01, 생일, 맞이, 강원도, 여행, 의, 시작, 후후, 제일, 기대하던,...","[(04, Number), (01, Number), (생일, Noun), (맞이, ..."
1,호텔 먼저 들어가서 체크인 선착순 티켓 받아오고 밥먹으러 고고싱 하마식당 웨이팅 걸...,0,"[호텔, 먼저, 들어가서, 체크, 인, 선착순, 티켓, 받아오고, 밥, 먹으러, 고...","[(호텔, Noun), (먼저, Noun), (들어가다, Verb), (체크, No..."
2,진짜 바다도 너무 푸르고 그저 행복 잠깐 기다리면서 애들 찍어봤는데 뭔가 배경도 괜...,0,"[진짜, 바다, 도, 너무, 푸르고, 그저, 행복, 잠깐, 기다리면서, 애, 들, ...","[(진짜, Noun), (바다, Noun), (도, Josa), (너무, Adver..."
3,근데 진짜 날씨가 너무 좋아서ㅠㅠㅠ 완벽했던 여행 바다가 저리 푸르고 하늘은 구름한...,0,"[근데, 진짜, 날씨, 가, 너무, 좋아서, ㅠㅠㅠ, 완벽했던, 여행, 바다, 가,...","[(근데, Adverb), (진짜, Noun), (날씨, Noun), (가, Jos..."
4,그리고 바닷가에서 벗어나니까 막 춥진 않은 날씨라서 식당 앞에서 기다리고 딱 좋음,0,"[그리고, 바닷가, 에서, 벗어나니까, 막, 춥진, 않은, 날씨, 라서, 식당, 앞...","[(그리고, Conjunction), (바닷가, Noun), (에서, Josa), ..."
...,...,...,...,...
821100,분석글은 저의 투자시점과 생각이고 흐름만 파악하는 용도이지 주식을 추천하는건 아닙니다,18280,"[분석, 글, 은, 저, 의, 투자, 시점, 과, 생각, 이고, 흐름, 만, 파악,...","[(분석, Noun), (글, Noun), (은, Josa), (저, Noun), ..."
821101,참고 만 하시고 투자손실의 책임은 본인에 있습니다,18280,"[참고, 만, 하시고, 투자, 손실, 의, 책임, 은, 본인, 에, 있습니다]","[(참고, Noun), (만, Noun), (하다, Verb), (투자, Noun)..."
821102,같이보면 좋은 포스팅 국내주식 사료 관련주 안녕하세요 여러분 주말에 푹쉬고 헬요일 ...,18280,"[같이, 보면, 좋은, 포스팅, 국내, 주식, 사료, 관련, 주, 안녕하세요, 여러...","[(같이, Adverb), (보다, Verb), (좋다, Adjective), (포..."
821103,오늘 정리해볼 테마는 사료관련주입니다 사료 m blog naver com 임산부커피...,18280,"[오늘, 정리, 해볼, 테마, 는, 사료, 관련, 주입, 니, 다, 사료, m, b...","[(오늘, Noun), (정리, Noun), (해보다, Verb), (테마, Nou..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_4월_형태소분석.txt", index=False, encoding='UTF-8')